In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, KFold
import wandb
from collections import namedtuple
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode
from sklearn import svm

In [2]:
wandb.login()

wandb: Currently logged in as: zongl (csc240_lztp). Use `wandb login --relogin` to force relogin


True

In [3]:
# rename columns
def remove_rename(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(["peak.1", "gene.1"], axis=1)

    num_cells = (len(df.columns)-4) // 2
    new_cols = ['peak', 'gene', 'Pair', 'is_pair'] + ["atac." + str(x) for x in range(num_cells)] + ["rna." + str(x) for x in range(num_cells)]
    df = df.rename(columns=dict(zip(df.columns, new_cols)))
    return df

def set_labels(df: pd.DataFrame) -> pd.DataFrame:
    df["is_pair"] = df["is_pair"].apply(lambda x: 1 if x else 0)
    return df
    
def normalize(df: pd.DataFrame) -> pd.DataFrame:
    # only counts
    num_df = df[df.columns[4:]]
    
    # Summing each column (sample) to get library sizes
    library_sizes = num_df.sum(axis=0)
    
    # Normalizing to CPM
    cpm_df = num_df.div(library_sizes, axis=1) * 10**6
    
    # If you want to log-transform the CPM data
    log_cpm_df = cpm_df.apply(lambda x: np.log2(x + 1))

    return pd.concat([df[df.columns[:4]], log_cpm_df], axis=1)

In [ ]:
# use only features given
DATA_PATH = "../data/Tab_delimited_text/"
train_df = pd.read_csv(DATA_PATH + "train.csv")
test_df = pd.read_csv(DATA_PATH + "test.csv")

train_df = remove_rename(train_df)
train_df = set_labels(train_df)
train_df = normalize(train_df)

test_df = remove_rename(test_df)
test_df = set_labels(test_df)
test_df = normalize(test_df)

# split features
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=0)

# split x, y
X_train = train_df[train_df.columns[4:]]
y_train = train_df["is_pair"]

X_val = val_df[val_df.columns[4:]]
y_val = val_df["is_pair"]

X_test = test_df[test_df.columns[4:]]

In [4]:
# or use features from features.ipynb
train_df = pd.read_csv("../data/train_features_3.csv")
test_df = pd.read_csv("../data/test_features_3.csv")

peak_score = "peak_tf_score"

# Create interaction features, scale features
for df in [train_df, test_df]:
    # Initialize the scaler
    scaler = StandardScaler()
    
    # Scale the 'peak_tf_score' column
    df['correlation'] = scaler.fit_transform(df[['correlation']])
    
    # Scale the 'distance' column
    # train_df['distance'] = np.log10(train_df['distance'])
    df['distance'] = scaler.fit_transform(df[['distance']])
    
    # Scale the 'peak_tf_score' column
    df[peak_score] = scaler.fit_transform(df[[peak_score]])
    
    df['interaction_c_d'] = df['correlation'] * df['distance']
    df['interaction_d_s'] = df['distance'] * df[peak_score]
    df['interaction_c_s'] = df['correlation'] * df[peak_score]
    df['interaction_c_d_s'] = df['correlation'] * df['distance'] * df[peak_score]

# split features
# train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=0)

features = ["correlation", "distance", peak_score, "interaction_c_d", "interaction_d_s", "interaction_c_s", "interaction_c_d_s"] + [f"pc_{x+1}" for x in range(20)]
# features= ["correlation", "distance"] + [f"pc_{x+1}" for x in range(20)]
# features = ["distance", peak_score, "interaction_c_d", "interaction_d_s", "interaction_c_s", "interaction_c_d_s"] + [f"pc_{x+1}" for x in range(20)]

# split x, y
X_train = train_df[features]
y_train = train_df["is_pair"]

# X_val = val_df[features]
# y_val = val_df["is_pair"]

X_test = test_df[features]

In [159]:
# parameter sweep
# sweep_configuration = {
#     "method": "bayes",
#     "name": "contrastive",
#     "metric": {"goal": "maximize", "name": "val_mcc"},
#     "parameters": {
#         'max_depth': {"min": 3, "max": 18, "distribution": "q_uniform"},
#         'gamma': {"min": 1, "max": 9, "distribution": "uniform"},
#         # 'reg_alpha': {"min":40, "max": 180, "distribution": "q_uniform"},
#         # 'reg_lambda': {"min": 0.0, "max": 1.0},
#         'colsample_bytree': {"min": 0.5, "max": 1.0},
#         'subsample': {"min": 0.5, "max": 1.0},
#         # 'min_child_weight': {"min": 0, "max": 10, "distribution": "q_uniform"},
#         'n_estimators': {"min": 2, "max": 10},
#         'learning_rate': {"value": 1},
#         'seed': {"value": 0}
#     },
# }
sweep_configuration = {
    "method": "bayes",
    "name": "contrastive",
    "metric": {"goal": "maximize", "name": "val_mcc"},
    "parameters": {
        'max_depth': {"min": 1, "max": 10, "distribution": "q_uniform"},
        'gamma': {"min": 1, "max": 9, "distribution": "uniform"},
        'reg_alpha': {"min":40, "max": 180, "distribution": "q_uniform"},
        'reg_lambda': {"min": 0.0, "max": 1.0},
        'colsample_bytree': {"min": 0.5, "max": 1.0},
        'subsample': {"min": 0.5, "max": 1.0},
        'min_child_weight': {"min": 0, "max": 10, "distribution": "q_uniform"},
        'n_estimators': {"values": [10, 50, 100, 250]},
        'learning_rate': {"values": [0.001, 0.01, 0.1, 1]},
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="hackathon-2024")

Create sweep with ID: 622kdym6
Sweep URL: https://wandb.ai/csc240_lztp/hackathon-2024/sweeps/622kdym6


In [160]:
# define metrics
metrics = [
    ("f1", f1_score),
    ("acc", accuracy_score),
    ("mcc", matthews_corrcoef),
]

In [161]:
def train(use_wandb=True, params=None):
    if use_wandb:
        wandb.init()
        config = wandb.config
    else:
        Config = namedtuple("configuration", list(params.keys()))
        config = Config(**params)
    bst = XGBClassifier(max_depth=config.max_depth,
                        gamma=config.gamma,
                        reg_alpha=config.reg_alpha,
                        reg_lambda=config.reg_lambda,
                        colsample_bytree=config.colsample_bytree,
                        subsample=config.subsample,
                        min_child_weight=config.min_child_weight,
                        n_estimators=config.n_estimators,
                        learning_rate=config.learning_rate,
                        objective="binary:hinge")

    # Define the number of folds
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

    train_metric_fold = {
        "f1": [],
        "acc": [],
        "mcc": [],
    }
    test_metric_fold = {
        "f1": [],
        "acc": [],
        "mcc": [],
    }
    for train_index, val_index in kf.split(X_train):
        # Split the data into training and validation sets
        X_train_fold, X_val_fold = X_train.to_numpy()[train_index], X_train.to_numpy()[val_index]
        y_train_fold, y_val_fold = y_train.to_numpy()[train_index], y_train.to_numpy()[val_index]
    
        bst = XGBClassifier(n_estimators=100,
                          gamma=1,
                          max_depth=1,
                          learning_rate=1,
                          subsample=None,
                          colsample_bytree=None,
                          objective="binary:hinge")
        
        bst.fit(X_train_fold, y_train_fold)
        
        y_pred_val = bst.predict(X_val_fold)
        y_pred_train = bst.predict(X_train_fold)
    
        for name, metric in metrics:
            train_metric_fold[name].append(metric(y_train_fold, y_pred_train))
            test_metric_fold[name].append(metric(y_val_fold, y_pred_val))
            
    if use_wandb:
        wandb.log({"val_" + metricname: np.mean(metriclist) for metricname, metriclist in test_metric_fold.items()})
        wandb.log({"train_" + metricname: np.mean(metriclist) for metricname, metriclist in train_metric_fold.items()})
    else:
        print({f"val_{metricname}": f"{np.mean(metriclist):.5f} ({np.std(metriclist):.5f})" for metricname, metriclist in test_metric_fold.items()})
        print({f"train_{metricname}": f"{np.mean(metriclist):.5f} ({np.std(metriclist):.5f})" for metricname, metriclist in train_metric_fold.items()})

    return config

In [163]:
wandb.agent(sweep_id, function=train, count=30)

wandb: Agent Starting Run: rapawzf8 with config:
wandb: 	colsample_bytree: 0.906918228152246
wandb: 	gamma: 2.611809828912506
wandb: 	learning_rate: 1
wandb: 	max_depth: 5
wandb: 	min_child_weight: 4
wandb: 	n_estimators: 50
wandb: 	reg_alpha: 58
wandb: 	reg_lambda: 0.21082387664922964
wandb: 	subsample: 0.6982488103572396


train_acc,▁
train_f1,▁
train_mcc,▁
val_acc,▁
val_f1,▁
val_mcc,▁
train_acc,1.0
train_f1,1.0
train_mcc,1.0
val_acc,0.99333
val_f1,0.99347


wandb: Agent Starting Run: rrziu5fy with config:
wandb: 	colsample_bytree: 0.9191065422636928
wandb: 	gamma: 1.0622897722836724
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 7
wandb: 	min_child_weight: 4
wandb: 	n_estimators: 50
wandb: 	reg_alpha: 67
wandb: 	reg_lambda: 0.4381598974442754
wandb: 	subsample: 0.5812351384309866


train_acc,▁
train_f1,▁
train_mcc,▁
val_acc,▁
val_f1,▁
val_mcc,▁
train_acc,1.0
train_f1,1.0
train_mcc,1.0
val_acc,0.99333
val_f1,0.99347


wandb: Agent Starting Run: yq025dgn with config:
wandb: 	colsample_bytree: 0.7459326978277444
wandb: 	gamma: 4.953478561819055
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 2
wandb: 	min_child_weight: 10
wandb: 	n_estimators: 250
wandb: 	reg_alpha: 129
wandb: 	reg_lambda: 0.865095263878509
wandb: 	subsample: 0.9809493786426202


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: xn905yju with config:
wandb: 	colsample_bytree: 0.5736568365700225
wandb: 	gamma: 6.115210720634398
wandb: 	learning_rate: 0.001
wandb: 	max_depth: 10
wandb: 	min_child_weight: 9
wandb: 	n_estimators: 100
wandb: 	reg_alpha: 57
wandb: 	reg_lambda: 0.9387593690044832
wandb: 	subsample: 0.5226015564742974


train_acc,▁
train_f1,▁
train_mcc,▁
val_acc,▁
val_f1,▁
val_mcc,▁
train_acc,1.0
train_f1,1.0
train_mcc,1.0
val_acc,0.99333
val_f1,0.99347


wandb: Agent Starting Run: ihj89iz7 with config:
wandb: 	colsample_bytree: 0.6213121238461423
wandb: 	gamma: 8.612715100467309
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 6
wandb: 	min_child_weight: 9
wandb: 	n_estimators: 100
wandb: 	reg_alpha: 152
wandb: 	reg_lambda: 0.42865855473104153
wandb: 	subsample: 0.762447479958198


train_acc,▁
train_f1,▁
train_mcc,▁
val_acc,▁
val_f1,▁
val_mcc,▁
train_acc,1.0
train_f1,1.0
train_mcc,1.0
val_acc,0.99333
val_f1,0.99347


wandb: Sweep Agent: Waiting for job.
wandb: Ctrl + C detected. Stopping sweep.


# Evaluate Best Model

In [21]:
# parameters
params = {
    "n_estimators": 5,
    "gamma": 5.681,
    "max_depth": 12,
    "learning_rate": 1,
    "subsample": 0.883,
    "colsample_bytree": 0.795,
    "objective": "binary:logistic",
    "seed": 0
}

In [33]:
model = train(use_wandb=False, params=params)

{'val_f1': 0.736842105263158, 'val_acc': 0.75, 'val_mcc': 0.4991650425568579}
{'train_f1': 0.943089430894309, 'train_acc': 0.9416666666666667, 'train_mcc': 0.8837842790654808}


In [37]:
# make predictions on test set
y_pred = model.predict(X_test)

In [60]:
# generate merge
merge_df = test_df[["Pair"]].copy(deep=True)
merge_df["pred"] = pd.Series(y_pred)

# update predictions
pred_df = pd.read_csv("../prediction/prediction_templ.csv")
merged_df = pred_df.merge(merge_df, on="Pair")[["peak", "gene", "Pair", "pred"]].rename(columns={"pred": "Peak2Gene"})
merged_df["Peak2Gene"] = merged_df["Peak2Gene"].apply(lambda x: "TRUE" if x else "FALSE")

In [62]:
merged_df.head()

,peak,gene,Pair,Peak2Gene
0,chr1-109466154-109467690,SORT1,chr1-109466154-109467690_SORT1,FALSE
1,chr1-116762742-116771198,CD2,chr1-116762742-116771198_CD2,TRUE
2,chr1-1245493-1248050,SDF4,chr1-1245493-1248050_SDF4,FALSE
3,chr1-1330394-1334148,MRPL20,chr1-1330394-1334148_MRPL20,FALSE
4,chr1-151939435-151947234,S100A10,chr1-151939435-151947234_S100A10,TRUE


In [64]:
merged_df.to_csv("../prediction/prediction.csv", index=False)

# with features only

## bagged model

In [110]:
def train_bagged_xgboost(X_train, y_train, num_models=5, subsample_size=0.8):
    models = []
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    for _ in range(num_models):
        # Bootstrap sampling
        indices = np.random.choice(range(X_train.shape[0]), size=int(subsample_size * X_train.shape[0]), replace=True)
        X_subset, y_subset = X_train[indices], y_train[indices]
        
        # Train model
        model = XGBClassifier(n_estimators=10,
                                  gamma=1,
                                  max_depth=3,
                                  learning_rate=1,
                                  subsample=None,
                                  colsample_bytree=None,
                                  seed=0,
                                  objective="binary:logistic")
        model.fit(X_subset, y_subset)
        models.append(model)
    return models

def bagged_predict(models, X_test):
    preds = np.column_stack([model.predict(X_test) for model in models])
    return mode(preds, axis=1).mode

In [114]:
models = train_bagged_xgboost(X_train, y_train, num_models=31, subsample_size=0.6)
y_pred_val = bagged_predict(models, X_val)
y_pred_train = bagged_predict(models, X_train)

print({"val_" + metricname: metric(y_val, y_pred_val) for metricname, metric in metrics})
print({"train_" + metricname: metric(y_train, y_pred_train) for metricname, metric in metrics})

{'val_f1': 0.9824561403508771, 'val_acc': 0.9833333333333333, 'val_mcc': 0.9671322699539122}
{'train_f1': 0.995850622406639, 'train_acc': 0.9958333333333333, 'train_mcc': 0.9917011013305196}


## baseline model

In [153]:
# Define the number of folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

train_metric_fold = {
    "f1": [],
    "acc": [],
    "mcc": [],
}
test_metric_fold = {
    "f1": [],
    "acc": [],
    "mcc": [],
}
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train.to_numpy()[train_index], X_train.to_numpy()[val_index]
    y_train_fold, y_val_fold = y_train.to_numpy()[train_index], y_train.to_numpy()[val_index]

    bst = XGBClassifier(n_estimators=100,
                      gamma=1,
                      max_depth=1,
                      learning_rate=1,
                      subsample=None,
                      colsample_bytree=None,
                      objective="binary:hinge")
    
    bst.fit(X_train_fold, y_train_fold)
    
    y_pred_val = bst.predict(X_val_fold)
    y_pred_train = bst.predict(X_train_fold)

    for name, metric in metrics:
        train_metric_fold[name].append(metric(y_train_fold, y_pred_train))
        test_metric_fold[name].append(metric(y_val_fold, y_pred_val))
        

print({f"val_{metricname}": f"{np.mean(metriclist):.5f} ({np.std(metriclist):.5f})" for metricname, metriclist in test_metric_fold.items()})
print({f"train_{metricname}": f"{np.mean(metriclist):.5f} ({np.std(metriclist):.5f})" for metricname, metriclist in train_metric_fold.items()})

{'val_f1': '0.99347 (0.00809)', 'val_acc': '0.99333 (0.00816)', 'val_mcc': '0.98669 (0.01630)'}
{'train_f1': '1.00000 (0.00000)', 'train_acc': '1.00000 (0.00000)', 'train_mcc': '1.00000 (0.00000)'}


## generate predictions

In [5]:
bst = XGBClassifier(n_estimators=10,
                      gamma=1,
                      max_depth=1,
                      learning_rate=1,
                      subsample=None,
                      colsample_bytree=None,
                      objective="binary:hinge")
    
bst.fit(X_train, y_train)
bst.get_booster().get_score()

{'correlation': 4.0}

In [6]:
# make predictions on test set
y_pred = bst.predict(X_test)

In [7]:
# generate merge
merge_df = test_df[["Pair"]].copy(deep=True)
merge_df["pred"] = pd.Series(y_pred)

# update predictions
pred_df = pd.read_csv("../prediction/prediction_templ.csv")
merged_df = pred_df.merge(merge_df, on="Pair")[["peak", "gene", "Pair", "pred"]].rename(columns={"pred": "Peak2Gene"})
merged_df["Peak2Gene"] = merged_df["Peak2Gene"].apply(lambda x: "TRUE" if x else "FALSE")

In [8]:
merged_df.to_csv("../prediction/prediction.csv", index=False)

In [9]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])